In [4]:
import getpass
from datetime import datetime
from math import prod, pow
import random
import string

# Function to generate Fibonacci numbers up to n terms
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

# Function to map each character to its corresponding Fibonacci number
def map_to_fibonacci(text):
    fib_seq = fibonacci(len(text))  # Generate Fibonacci numbers for the length of the text
    fib_map = {char: fib_seq[i] for i, char in enumerate(text)}
    return fib_map

# Function to calculate harmonic mean
def harmonic_mean(numbers):
    return len(numbers) / sum(1 / num for num in numbers if num != 0)  # Avoid division by zero

# Function to calculate geometric mean
def geometric_mean(numbers):
    product = prod(numbers)
    return pow(product, 1 / len(numbers))

# Function to calculate proportional mean between HM and GM
def proportional_mean(hm, gm):
    return (hm + gm) / 2

# Function to extract digits from proportional mean
def extract_digits(number):
    # Convert the proportional mean to a string and extract digits
    return ''.join([char for char in str(number) if char.isdigit()])

# Function to ensure at least 25% of the characters are from a specified group
def enforce_min_percentage(otp, required_chars, total_length, percentage):
    required_count = max(1, round(total_length * percentage))  # Minimum 25%
    current_count = sum(1 for char in otp if char in required_chars)
    while current_count < required_count:
        replace_index = random.randint(0, total_length - 1)
        if otp[replace_index] not in required_chars:
            otp = otp[:replace_index] + random.choice(required_chars) + otp[replace_index+1:]
            current_count += 1
    return otp

# Function to generate OTP based on selected parameters
def generate_otp_from_pm(pm_digits, otp_length, option, login, password):
    otp = ''
    characters = pm_digits  # Start with digits from proportional mean

    # Rule for option 1: Only numbers
    if option == 1:
        otp = ''.join(random.choices(pm_digits, k=otp_length))

    # Rule for option 2: Numbers and small letters from login/password
    elif option == 2:
        lowercase_letters = ''.join([char.lower() for char in login + password if char.isalpha()])
        characters += lowercase_letters
        otp = ''.join(random.choices(characters, k=otp_length))

    # Rule for option 3: Numbers, small and large letters, minimum 25% large letters
    elif option == 3:
        letters = ''.join([char for char in login + password if char.isalpha()])
        characters += letters
        otp = ''.join(random.choices(characters, k=otp_length))
        otp = enforce_min_percentage(otp, string.ascii_uppercase, otp_length, 0.25)  # Ensure 25% uppercase letters

    # Rule for option 4: Numbers, letters, special characters, minimum 25% large letters and special characters
    elif option == 4:
        letters = ''.join([char for char in login + password if char.isalpha()])
        special_chars_in_login_password = ''.join([char for char in login + password if char in string.punctuation])

        # Add random special characters if not enough in login/password
        if len(special_chars_in_login_password) < 2:
            special_chars = ''.join(random.choices(string.punctuation, k=2))
        else:
            special_chars = special_chars_in_login_password

        characters += letters + special_chars
        otp = ''.join(random.choices(characters, k=otp_length))
        otp = enforce_min_percentage(otp, string.ascii_uppercase, otp_length, 0.25)  # Ensure 25% uppercase letters
        otp = enforce_min_percentage(otp, string.punctuation, otp_length, 0.25)  # Ensure 25% special characters

    return otp

# Ask for login
login = input("Enter your login: ")

# Ask for password securely
password = getpass.getpass("Enter your password: ")

# Get current date and time
current_time = datetime.now()

# Format the date and time to string with seconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

# Map login, password, and formatted time to Fibonacci numbers
login_fib = map_to_fibonacci(login)
password_fib = map_to_fibonacci(password)
datetime_fib = map_to_fibonacci(formatted_time)

# Combine all Fibonacci numbers from login, password, and datetime
all_fib_numbers = list(login_fib.values()) + list(password_fib.values()) + list(datetime_fib.values())

# Filter out zeroes, as harmonic and geometric means require positive values
non_zero_fib_numbers = [num for num in all_fib_numbers if num != 0]

# Calculate the harmonic mean
hm = harmonic_mean(non_zero_fib_numbers)

# Calculate the geometric mean
gm = geometric_mean(non_zero_fib_numbers)

# Calculate the proportional mean
pm = proportional_mean(hm, gm)

# Extract digits from the proportional mean
pm_digits = extract_digits(pm)

print(f"\nFibonacci mapping for your login, password, and datetime:")
print(f"Fibonacci numbers: {all_fib_numbers}")
print(f"Harmonic mean: {hm}")
print(f"Geometric mean: {gm}")
print(f"Proportional mean: {pm}")
print(f"Digits extracted from proportional mean: {pm_digits}")

# OTP Password generation menu
if len(pm_digits) == 0:
    print("Error: No digits available from proportional mean.")
else:
    print("\n--- OTP Password Generation Menu ---")

    otp_length = int(input("1. How many characters in the OTP password? "))

    print("2. OTP password parameters:")
    print("1 - Only numbers")
    print("2 - Numbers and small letters")
    print("3 - Numbers, small and large letters")
    print("4 - Numbers, small and large letters, special characters")

    otp_option = int(input("Select an option: "))

    # Generate and display the OTP based on the selected option
    otp_password = generate_otp_from_pm(pm_digits, otp_length, otp_option, login, password)

    print(f"\nYour OTP password is: {otp_password}")


Enter your login: dommovoy
Enter your password: ··········

Fibonacci mapping for your login, password, and datetime:
Fibonacci numbers: [0, 8, 2, 5, 13, 0, 1, 1, 2, 3, 5, 34, 13, 21, 89, 144, 2, 13, 8, 377, 55, 987, 1597, 2584]
Harmonic mean: 4.5528157560052795
Geometric mean: 20.405054307950746
Proportional mean: 12.478935031978013
Digits extracted from proportional mean: 12478935031978013

--- OTP Password Generation Menu ---
1. How many characters in the OTP password? 6
2. OTP password parameters:
1 - Only numbers
2 - Numbers and small letters
3 - Numbers, small and large letters
4 - Numbers, small and large letters, special characters
Select an option: 4

Your OTP password is: @L3<18
